# State Farm Distracted Driver Detection


[State Farm Distracted Driver Detection](https://www.kaggle.com/c/state-farm-distracted-driver-detection#evaluation)

## Action Plan
### 1. Data Preparation and Preprocessing
### 2. Finetune and Train Model
### 3. Generate and Validate Predictions 
### 4. Submit predictions to Kaggle

## 2. Finetune and Train Model

In [1]:
%cd "~/kaggle/state-farm-driver-detection/code"
%pwd

/home/ubuntu/kaggle/state-farm-driver-detection/code


'/home/ubuntu/kaggle/state-farm-driver-detection/code'

In [2]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
CODE_HOME_DIR = current_dir
DATA_HOME_DIR = CODE_HOME_DIR + '/../input/'
print(CODE_HOME_DIR)
print(DATA_HOME_DIR)

/home/ubuntu/kaggle/state-farm-driver-detection/code
/home/ubuntu/kaggle/state-farm-driver-detection/code/../input/


In [3]:
#import modules
#from importlib import reload

import utils
from utils import *

import vgg16bn_ted
from vgg16bn_ted import Vgg16BN; 

%matplotlib inline

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Setup 

#### Setup Paths

In [4]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' # + '/sample/' 
test_path = path + '/test/'
results_path = path + '/results/'
train_path = path + '/train/'
valid_path = path + '/valid/'
model_path = path + '/models/'
if not os.path.exists(model_path): os.mkdir(model_path)

/home/ubuntu/kaggle/state-farm-driver-detection/input


## First training

In [5]:
vgg = Vgg16BN(size=(256, 256))
#vgg = Vgg16BN()
model = vgg.model

In [6]:
batch_size=64

In [8]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, shuffle=False, batch_size=batch_size)
vgg.finetune(batches)

Found 17940 images belonging to 10 classes.
Found 4484 images belonging to 10 classes.


In [9]:
vgg.fit(batches, val_batches, epochs=2)

Epoch 1/2
 - 346s - loss: 1.1656 - acc: 0.6431 - val_loss: 0.1348 - val_acc: 0.9674
Epoch 2/2
 - 337s - loss: 0.4400 - acc: 0.8605 - val_loss: 0.0854 - val_acc: 0.9784


In [10]:
model.optimizer.lr = 0.01

In [11]:
vgg.fit(batches, val_batches, epochs=5)

Epoch 1/5
 - 336s - loss: 0.3565 - acc: 0.8824 - val_loss: 0.0634 - val_acc: 0.9837
Epoch 2/5
 - 337s - loss: 0.3394 - acc: 0.8917 - val_loss: 0.0738 - val_acc: 0.9804
Epoch 3/5
 - 337s - loss: 0.3223 - acc: 0.8993 - val_loss: 0.0566 - val_acc: 0.9855
Epoch 4/5
 - 337s - loss: 0.3240 - acc: 0.8959 - val_loss: 0.0637 - val_acc: 0.9819
Epoch 5/5
 - 337s - loss: 0.3366 - acc: 0.8988 - val_loss: 0.0529 - val_acc: 0.9868


In [12]:
model.save_weights(model_path+'vgg16bn-first1.h5')